<a href="https://colab.research.google.com/github/raffieeey/MasterResearchAutoML/blob/master/02A_H2OAutoml_Regres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing dependecies for H2O automl

In [1]:
!pip install pmlb
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install pytictoc

  Created wheel for pmlb: filename=pmlb-0.3-cp36-none-any.whl size=11918 sha256=09e90ca85e977b672969a6ffe5b2c46deb3048e4ac23081b547639c9969a224c
  Stored in directory: /root/.cache/pip/wheels/5a/c7/a4/be59e63a2cb56f6c58f068305b95c212e0aac1a930fd77d6b0
Successfully built pmlb


In [0]:
#!pip uninstall h2o -y

## Installing H2O

In [3]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 123.6MB 133kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.5-py2.py3-none-any.whl size=123672284 sha256=9429089d5def1f5e780367c34b4edf5ad554eeb6bdde2beddbe36c14008bbc19
  Stored in directory: /root/.cache/pip/wheels/c9/d2/24/eeb5f07ff6826ea23325338cd3fb5754b99592a3f2b4e67f27
Successfully built h2o


## Getting the number of Cores

In [4]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits 

##Starting H2O

In [0]:
import h2o

In [6]:
h2o.init(nthreads=2,
                 min_mem_size="2048M",
                 max_mem_size="2048M")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmptpdk6ygy
  JVM stdout: /tmp/tmptpdk6ygy/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmptpdk6ygy/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.5
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_unknownUser_mo3tpn
H2O cluster total nodes:,1
H2O cluster free memory:,2 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


## Creating all the metrics for Regression algorithm

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
import math
import numpy as np

class MetricsCalc:

    def __init__(self, truth, predictions):
        self.truth = truth.as_data_frame()
        self.predictions = predictions
        self._type = 'regression'

    def mae(self):
        return float(mean_absolute_error(self.truth, self.predictions))

    def mse(self):
        return float(mean_squared_error(self.truth, self.predictions))

    def msle(self):
      try:
        return float(mean_squared_log_error(self.truth, self.predictions))
      except:
        return np.nan
    
    def rmse(self):
        return math.sqrt(self.mse())

    def rmsle(self):
        return math.sqrt(self.msle())

    def r2(self):
        return float(r2_score(self.truth, self.predictions))

## Running H2O autoML with 
- 120 seconds
- Seed 69
- 2 Cores
- 2GB memory

- Tested for metrics: 
> - Deviance
> - Mean Squared error
> - Root Mean Squared error
> - Mean Absolute error
> - Root Mean Squared Logarithmic Error

- Recorded for metrics: 
> - Mean Absolute error
> - Mean Squared error
> - Mean Squared Logarithmic Error
> - Root Mean Squared error
> - Root Mean Squared Logarithmic Error
> - r2

In [42]:
from pmlb import classification_dataset_names, regression_dataset_names,fetch_data
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from pytictoc import TicToc
import pandas as pd

df_result = pd.DataFrame(columns=["dataset_name", "metrics_recorded", "MAE", "MSE","MSLE", "RMSE", "RMSLE", "r2"])
preds = {}
top_algo = {}
t = TicToc()


sel_clss_dtst = ['adult','agaricus-lepiota', 'churn', 'nursery', 'satimage','texture']
sel_rgrs_dtst = ['294_satellite_image','218_house_8L', '227_cpu_small', '503_wind', '344_mv','215_2dplanes']

ix = 0

for rgrs_dtst in sel_rgrs_dtst:
  for metrics_recorded in ['RMSLE', 'MAE', 'RMSE', 'MSE', 'deviance']:
        df_result.loc[ix,'dataset_name'] = rgrs_dtst
        df_result.loc[ix,'metrics_recorded'] = metrics_recorded
        print("_"*80)
        print(f"*** Dataset Name: {rgrs_dtst} ***")
        X, y = fetch_data(rgrs_dtst, return_X_y=True)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=69)
        X_train = h2o.H2OFrame(X_train)
        X_test = h2o.H2OFrame(X_test)
        y_train = h2o.H2OFrame(y_train)
        y_test = h2o.H2OFrame(y_test)
        aml = H2OAutoML(seed=69, max_runtime_secs=120,sort_metric=metrics_test)
        t.tic()
        aml.train(x=X_train.columns, y=X_train.cbind(y_train)[:,-1].columns[0], training_frame=X_train.cbind(y_train))
        t.toc()    
        lb = aml.leaderboard
        top_algo[rgrs_dtst] = lb.head(5)
        print("*** Top algorithm: ***", lb.head(5))
        preds[rgrs_dtst] = aml.leader.predict(X_test).as_data_frame()
        p = MetricsCalc(y_test,preds[rgrs_dtst])
        df_result.loc[ix,'MAE'] = p.mae()
        df_result.loc[ix,'MSE'] = p.mse()
        df_result.loc[ix,'MSLE'] = p.msle()
        df_result.loc[ix,'RMSE'] = p.rmse()
        df_result.loc[ix,'RMSLE'] = p.rmsle()
        df_result.loc[ix,'r2'] = p.r2()
        ix+=1

________________________________________________________________________________
*** Dataset Name: 294_satellite_image ***
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Elapsed time is 122.699425 seconds.
*** Top algorithm: *** 

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20190925_062059,0.435182,0.659684,0.435182,0.345599,0.139374
XGBoost_1_AutoML_20190925_062059,0.448831,0.669948,0.448831,0.346144,0.141641
XGBoost_2_AutoML_20190925_062059,0.441276,0.664286,0.441276,0.348864,0.140782
StackedEnsemble_BestOfFamily_AutoML_20190925_062059,0.448563,0.669749,0.448563,0.352014,0.141763
GBM_1_AutoML_20190925_062059,0.505172,0.710754,0.505172,0.377299,0.153004



stackedensemble prediction progress: |████████████████████████████████████| 100%
________________________________________________________________________________
*** Dataset Name: 294_satellite_image ***
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Elapsed time is 122.737213 seconds.
*** Top algorithm: *** 

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20190925_062303,0.43518,0.659682,0.43518,0.345559,0.139411
XGBoost_1_AutoML_20190925_062303,0.448831,0.669948,0.448831,0.346144,0.141641
XGBoost_2_AutoML_20190925_062303,0.441276,0.664286,0.441276,0.348864,0.140782
StackedEnsemble_BestOfFamily_AutoML_20190925_062303,0.448106,0.669407,0.448106,0.351698,0.141756
GBM_1_AutoML_20190925_062303,0.500762,0.707645,0.500762,0.375574,0.152452



stackedensemble prediction progress: |████████████████████████████████████| 100%
________________________________________________________________________________
*** Dataset Name: 294_satellite_image ***
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Elapsed time is 122.383861 seconds.
*** Top algorithm: *** 

...


xgboost prediction progress: |████████████████████████████████████████████| 100%
________________________________________________________________________________
*** Dataset Name: 215_2dplanes ***
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Elapsed time is 121.909246 seconds.
*** Top algorithm: *** 

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20190925_071924,1.09062,1.04433,1.09062,0.832636,nan



xgboost prediction progress: |████████████████████████████████████████████| 100%
________________________________________________________________________________
*** Dataset Name: 215_2dplanes ***
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Elapsed time is 121.856833 seconds.
*** Top algorithm: *** 

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20190925_072128,1.09245,1.04521,1.09245,0.833416,nan



xgboost prediction progress: |████████████████████████████████████████████| 100%


## Looking at the predictions

In [46]:
preds

{'215_2dplanes':        predict
 0    -1.157418
 1    -0.676852
 2    -6.867717
 3    -2.746616
 4    -4.920313
 5    -2.841637
 6    -3.757914
 7     6.851336
 8    -3.567621
 9     4.948258
 10    2.655684
 11    4.352353
 12   -0.044295
 13    1.007499
 14    2.568168
 15    2.943497
 16    3.223058
 17   -2.062057
 18    8.297565
 19   -4.553700
 20   -0.846332
 21    6.337327
 22    4.461475
 23    4.843818
 24   -5.389386
 25   -0.107938
 26   -0.501622
 27    1.457880
 28   -2.748125
 29   -0.647506
 ...        ...
 4047  0.890552
 4048  1.065534
 4049 -3.622949
 4050 -1.098540
 4051 -3.802915
 4052 -2.576834
 4053 -0.393100
 4054 -0.957409
 4055  8.923175
 4056 -1.104411
 4057 -2.769584
 4058  0.824721
 4059 -2.991873
 4060  3.114316
 4061  0.787614
 4062  3.862248
 4063 -2.461978
 4064  2.561393
 4065 -3.917459
 4066 -2.627966
 4067  1.153885
 4068  3.534601
 4069 -4.683108
 4070 -5.076747
 4071  0.178314
 4072  3.851251
 4073  1.172846
 4074  0.703836
 4075  8.722471
 4076  0

## Saving the predictions

In [0]:
for i, val in enumerate(preds):
  preds[val].to_csv(f"{val}.csv")

## Looking at the AUC

In [44]:
df_result

,dataset_name,metrics_recorded,MAE,MSE,MSLE,RMSE,RMSLE,r2
0,294_satellite_image,RMSLE,0.340757,0.44905,0.018749,0.670112,0.136927,0.907185
1,294_satellite_image,MAE,0.340461,0.44923,0.0187606,0.670246,0.136969,0.907148
2,294_satellite_image,RMSE,0.340461,0.44923,0.0187606,0.670246,0.136969,0.907148
3,294_satellite_image,MSE,0.340962,0.448516,0.0187266,0.669713,0.136845,0.907295
4,294_satellite_image,deviance,0.340938,0.448842,0.0187389,0.669957,0.13689,0.907228
5,218_house_8L,RMSLE,15343.4,8.32224e+08,0.386895,28848.3,0.622009,0.678455
6,218_house_8L,MAE,15343.4,8.32224e+08,0.386895,28848.3,0.622009,0.678455
7,218_house_8L,RMSE,15343.4,8.32224e+08,0.386895,28848.3,0.622009,0.678455
8,218_house_8L,MSE,15343.4,8.32224e+08,0.386895,28848.3,0.622009,0.678455
9,218_house_8L,deviance,15343.4,8.32224e+08,0.386895,28848.3,0.622009,0.678455


## Looking at the mean per class error

## Looking at the top algorithm used by the AutoML

In [45]:
top_algo

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20190925_072128,1.09245,1.04521,1.09245,0.833416,nan


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20190925_063949,8.87088e+08,29784,8.87088e+08,15391.1,0.622207
XGBoost_2_AutoML_20190925_063949,8.89982e+08,29832.6,8.89982e+08,15440.5,0.621009
XGBoost_3_AutoML_20190925_063949,9.18089e+08,30300,9.18089e+08,15992.3,0.631602
StackedEnsemble_AllModels_AutoML_20190925_063949,2.62156e+09,51201.1,2.62156e+09,31192.2,0.911061
StackedEnsemble_BestOfFamily_AutoML_20190925_063949,2.74687e+09,52410.6,2.74687e+09,32066.7,0.92781


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20190925_065015,6.96306,2.63876,6.96306,1.83179,0.0909504
XGBoost_2_AutoML_20190925_065015,7.03657,2.65265,7.03657,1.83733,nan
StackedEnsemble_AllModels_AutoML_20190925_065015,7.01014,2.64767,7.01014,1.86436,nan
XGBoost_3_AutoML_20190925_065015,7.59275,2.75549,7.59275,1.94118,nan
StackedEnsemble_BestOfFamily_AutoML_20190925_065015,7.38521,2.71757,7.38521,1.95488,nan


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20190925_062915,0.435119,0.659635,0.435119,0.345608,0.139346
XGBoost_1_AutoML_20190925_062915,0.448831,0.669948,0.448831,0.346144,0.141641
XGBoost_2_AutoML_20190925_062915,0.441276,0.664286,0.441276,0.348864,0.140782
StackedEnsemble_BestOfFamily_AutoML_20190925_062915,0.448615,0.669787,0.448615,0.352136,0.141732
DRF_1_AutoML_20190925_062915,0.506311,0.711555,0.506311,0.377664,0.152054


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20190925_071108,0.112725,0.335746,0.112725,0.211459,nan


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20190925_070041,9.1998,3.03312,9.1998,2.31219,0.214304
StackedEnsemble_BestOfFamily_AutoML_20190925_070041,9.33088,3.05465,9.33088,2.32738,0.215564
XGBoost_3_AutoML_20190925_070041,9.46312,3.07622,9.46312,2.35013,0.216929
XGBoost_1_AutoML_20190925_070041,9.56284,3.09238,9.56284,2.3571,0.217846
XGBoost_2_AutoML_20190925_070041,9.51857,3.08522,9.51857,2.35735,0.217348


{'215_2dplanes': ,
 '218_house_8L': ,
 '227_cpu_small': ,
 '294_satellite_image': ,
 '344_mv': ,
 '503_wind': }